In [76]:
rdd=sc.parallelize([103]+[1,100,200])

In [77]:
type(rdd) #case sensitive
#lambda function programming
#lambda argument_list: expression 


pyspark.rdd.RDD

In [78]:
rdd.take(2)#must specify argument so shows limitied only

[103, 1]

In [79]:
rdd.collect() #shows entire data with exact structure

[103, 1, 100, 200]

In [80]:
#average
mean=rdd.sum()/float(rdd.count()) #float otherwise round off type problem -precison due to implict conversion cast to int
print(mean)

101.0

In [81]:
#median same as mean but extreme values less effect on it due to sorting
rdd.sortBy(lambda x:x).collect()    #sort the list and middle value if two numbers average it


[1, 100, 103, 200]

In [82]:
rdd.sortBy(lambda x:x).zipWithIndex().collect() #adding index to each entry

[(1, 0), (100, 1), (103, 2), (200, 3)]

In [83]:
sortedindex=rdd.sortBy(lambda x:x).zipWithIndex().map(lambda (value,key) : (key,value)) # value key only working in python 2.0
# if .collect() used at end it will display but type of object changing to list and count not working
print(sortedindex.take(10))
print(type(sortedindex))

[(0, 1), (1, 100), (2, 103), (3, 200)]
<class 'pyspark.rdd.PipelinedRDD'>


In [84]:
type(rdd.sortBy(lambda x:x).zipWithIndex().map(lambda (value,key) : (key,value)).collect())

list

In [85]:
cn=sortedindex.count()
if (cn%2 ==1):
    index=(cn-1)/2 #
    print sortedindex.lookup(index) #checks whats at that index from above
else:
    index1=(cn/2)-1
    index2=cn/2
    v1=sortedindex.lookup(index1)[0]
    v2=sortedindex.lookup(index2)[0] #lookup returns list to get rid of it
    print v1
    print v2
    print(((v1+v2)/2))
    #median far more resistant to outliers

100
103
101


In [182]:
#covariance by variance of different varaibles gives correlation
rddx=sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rddy=sc.parallelize([7,6,5,4,5,6,7,8,9,10])

In [183]:
mx=rddx.sum()/float(rddx.count())
my=rddy.sum()/float(rddy.count())

In [184]:
rddxy=rddx.zip(rddy) #for both accessing at a same time
rddxy.take(2)

[(1, 7), (2, 6)]

In [185]:
covxy=rddxy.map(lambda (x,y):(x-mx)*(y-my)).sum()/float(rddx.count())
covxy

3.65

In [186]:
from math import sqrt
n=rddxy.count()
sdx=sqrt(rddx.map(lambda x:pow(x-mx,2)).sum()/float(rddx.count()))
sdy=sqrt(rddy.map(lambda y:pow(y-my,2)).sum()/float(rddy.count()))
sdx

2.8722813232690143

In [187]:
corrxy=covxy/float(sdx*sdy)
corrxy
#correlation tells which colunks intersting are correlated to each other

0.709272912083725

In [188]:
import random
from pyspark.mllib.stat import Statistics
c1=sc.parallelize(range(100))
c2=sc.parallelize(range(100,200))
c3=sc.parallelize(list(reversed(range(100)))) #list plz check
c4=sc.parallelize(random.sample(range(100),100))


In [181]:
data=c1.zip(c2).zip(c3).zip(c4).map(lambda (((a,b),c),d):(a,b,c,d)).map(lambda (a,b,c,d):[a,b,c,d]) #nested tuple input to normal tuple output and need list for rdd as output
data.take(10)


[[0, 100, 99, 41],
 [1, 101, 98, 48],
 [2, 102, 97, 8],
 [3, 103, 96, 72],
 [4, 104, 95, 46],
 [5, 105, 94, 39],
 [6, 106, 93, 6],
 [7, 107, 92, 75],
 [8, 108, 91, 80],
 [9, 109, 90, 22]]

In [180]:
Statistics.corr(data)

array([[ 1.        ,  1.        , -1.        ,  0.04682868],
       [ 1.        ,  1.        , -1.        ,  0.04682868],
       [-1.        , -1.        ,  1.        , -0.04682868],
       [ 0.04682868,  0.04682868, -0.04682868,  1.        ]])